In [1]:
import sys
import numpy as np
sys.path.append("../utils")
from mujoco_parser import MuJoCoParserClass
from util import MultiSliderClass,rpy2r
print ("Ready.")

Ready.


In [2]:
xml_path = '../asset/kimlab_dualarms/scene_dualarms.xml'
env = MuJoCoParserClass(name='Dual Arms',rel_xml_path=xml_path,VERBOSE=False)
env.set_geom_color(
    rgba                = (0.2,0.2,1.0,0.9),
    body_names_to_color = [
        'robot1/end_link','robot1/index_L1','robot1/index_L2','robot1/index_tip',
        'robot1/middle_L1','robot1/middle_L2','robot1/middle_tip',
        'robot1/ring_L1','robot1/ring_L2','robot1/ring_tip',
        'robot1/pinky_L1','robot1/pinky_L2','robot1/pinky_tip',
        'robot1/thumb_L1','robot1/thumb_L2','robot1/thumb_tip'],
)
env.set_geom_color(
    rgba                = (1.0,0.2,0.2,0.9),
    body_names_to_color = [
        'robot2/end_link','robot2/index_L1','robot2/index_L2','robot2/index_tip',
        'robot2/middle_L1','robot2/middle_L2','robot2/middle_tip',
        'robot2/ring_L1','robot2/ring_L2','robot2/ring_tip',
        'robot2/pinky_L1','robot2/pinky_L2','robot2/pinky_tip',
        'robot2/thumb_L1','robot2/thumb_L2','robot2/thumb_tip'],
)
print ("Done.")

Done.


In [3]:
q = np.zeros(env.model.nq)
q[0] = np.pi/2
q[5] = np.pi/2
env.forward(q)

wrist_pos = env.get_p_joint('robot1/joint6')
# JOINT_NAMES = ['robot1/joint1','robot1/joint2','robot1/joint3','robot1/joint4','robot1/joint5','robot1/joint6']
JOINT_NAMES = ['robot1/joint1','robot1/joint3','robot1/joint5','robot1/joint6']
joint_idxs_arms = env.get_idxs_fwd(joint_names=JOINT_NAMES)

In [4]:
sliders = MultiSliderClass(
    n_slider      = 3,
    title         = 'Sliders for Dual-Arms Wrist Control',
    window_width  = 500,
    window_height = 800,
    x_offset      = 50,
    y_offset      = 100,
    slider_width  = 350,
    label_texts   = ['x','y','z'],
    slider_mins   = wrist_pos + [-1,-1,-1],
    slider_maxs   = wrist_pos + [1,1,1],
    slider_vals   = wrist_pos,
    resolution    = 0.01,
    VERBOSE       = False,
)

env.init_viewer(
    viewer_title='Dual-Arms',viewer_width=1200,viewer_height=800,
    viewer_hide_menus=True,
)
env.update_viewer(
    azimuth=180,distance=1.24,elevation=-32,lookat=[0.25,0.5,1.73],
    VIS_TRANSPARENT=True,
)
env.forward(q)

while env.is_viewer_alive():
    sliders.update()
    tp = sliders.get_slider_values()

    q_arms = env.get_qpos_joints(JOINT_NAMES)
    J_list,ik_err_list = [],[]
    ik_p_target = tp
    ik_R_target = np.eye(3)
    J,ik_err = env.get_ik_ingredients(body_name='robot1/end_link',p_trgt=ik_p_target,R_trgt=ik_R_target)
    dq = env.damped_ls(J,ik_err,stepsize=1,eps=1e-2,th=np.radians(1.0))
    q_arms += dq[joint_idxs_arms]
    q_arms = np.clip(q_arms,env.rev_joint_mins[joint_idxs_arms],env.rev_joint_maxs[joint_idxs_arms])
    env.forward(q=q_arms,joint_idxs=joint_idxs_arms)
    wp = env.get_p_joint('robot1/joint6')

    env.plot_T(p=tp,R=ik_R_target,PLOT_AXIS=True,axis_len=0.1,axis_width=0.005,
               label='Wrist Target: [%.2f %.2f %.2f]\n Wrist Position: [%.2f %.2f %.2f]\n Distance: %.4f' %
               (tp[0],tp[1],tp[2],wp[0],wp[1],wp[2],np.linalg.norm(tp-wp)))
    env.plot_T(p=env.get_p_body('robot1/end_link'),R=env.get_R_body('robot1/end_link'),PLOT_AXIS=True,axis_len=0.1,axis_width=0.005)
    env.render()

env.close_viewer()
sliders.close()

Pressed ESC
Quitting.
